In [ ]:
#Import thư viện và kết nối Neo4j
import csv
import os
from py2neo import Graph, Node
graph=Graph("bolt://localhost:7687",auth=("neo4j", "123456789"))


In [ ]:
#Load file movies.csv và tạo node Movie
def loadMovies():
    with open('data/ml-20m/movies.csv', encoding="latin-1") as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for i, row in enumerate(readCSV):
            createMovieNodes(row)

def createMovieNodes(row):
    movieData = parseRowMovie(row)
    id = movieData[0]
    title = movieData[1]
    mov = Node("Movie", id=id, title=title)
    graph.create(mov)

def parseRowMovie(row):
        id = row[0]
        title = row[1]
        return (id, title)
loadMovies()


In [ ]:
#Load file ratings.csv, tạo node User và tạo quan hệ User - RATED - Movie
def createUserNodes(row):
    user = Node("User", id="User " + row[0])
    graph.merge(user, "User", "id")

def createRatingRelationship(row):
    ratingData = parseRowRatingRelationships(row)

    graph.run(
        'MATCH (u:User {id: $userId}), (m:Movie {id: $movieId}) CREATE (u)-[:RATED { rating: $rating, timestamp: $timestamp }]->(m)',
        userId=ratingData[0], movieId=ratingData[1], rating=ratingData[2], timestamp=ratingData[3])

def parseRowRatingRelationships(row):
    userId = "User " + row[0]
    movieId = row[1]
    rating = float(row[2])
    timestamp = row[3]
    return (userId, movieId, rating, timestamp)
def loadRatings():
    with open('data/ml-20m/ratings.csv', encoding="latin-1") as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for i,row in enumerate(readCSV):
            createUserNodes(row)
            createRatingRelationship(row)
loadRatings()


In [ ]:
# Tạo Index cho các đối tượng
graph.run('CREATE INDEX ON :User(id)')
graph.run('CREATE INDEX ON :Movie(id)')